In [1]:
from lib.header_notebook import *
import lib.OTCommon as OTCommon
import lib.SinkhornNP as Sinkhorn
import lib.LinHK as LinHK
import lib.LinW2 as LinW2

%matplotlib inline

# Compute HK tangent space embedding

In [2]:
# load reference measure, here we just take Euclidean mean
datRef=OTCommon.importMeasure("data/two_ellipses_rad0d25/mean.mat",totalMass=1.,keepZero=False)
muRef=datRef[1]
posRef=datRef[2].astype(np.double)

In [3]:
params={}
params["setup_HKMode"]=True
params["setup_HKScale"]=5
params["solver_errorGoal"]=1.E-4
params["aux_verbose"]=False
params["solver_epsStart"]=1E3
params["solver_epsTarget"]=0.1

In [4]:
# compute all tangent vectors and distances
tangentList=[]
distList=[]
for i in range(64):
    print(i)
    filename="data/two_ellipses_rad0d25/sample_{:03d}.mat".format(i)
    datSamp=OTCommon.importMeasure(filename,totalMass=1.,keepZero=False)
    muSamp=datSamp[1]
    posSamp=datSamp[2].astype(np.double)

    # solve HK transport
    value,pi=Sinkhorn.SolveHK(muRef,posRef,muSamp,posSamp,HKScale=params["setup_HKScale"],
            SinkhornError=params["solver_errorGoal"],
            epsTarget=params["solver_epsTarget"], epsInit=params["solver_epsStart"],
            returnSolver=False
            )    
    # from optimal coupling compute tangent vector data
    u0,u1,x1,refPerp,sampPerp=LinHK.extractMongeData(pi,muRef,muSamp,posSamp)
    v0,alpha0=LinHK.HKLog(posRef,u0,x1,u1,params["setup_HKScale"])
    tangentList.append((v0,alpha0,sampPerp))
    distList.append(value)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [5]:
import pickle
with open("experiments/two_ellipses_rad0d25_HK5/tangent.dat", 'wb') as f:
    pickle.dump([tangentList,distList],f,2)